# Azure Machine Learning Pipeline Example - v2 SDK

Sample notebook showcasing how to convert a python script (ex: `./scripts/custom_logic.py`) into a reusable Azure Machine Learning pipeline that can be triggered on a schedule.

Here, we convert the python script at `./scripts/custom_logic.py` into a component that can be included as a step in an AML pipeline. Pipelines can contain multiple steps with intermediate outputs at each step. In this example we execute all logic as part of a single step as a standalone component. Moreover, this step utilizes a custom environment which includes all necessary python packages and is defined in `./environment.yml`.

In [ ]:
# Import necessary packages and modules  
from azure.ai.ml import MLClient, Input, automl
from azure.ai.ml import command, Input
from azure.ai.ml.dsl import pipeline
from azure.ai.ml.entities import CommandJob, CommandJobLimits, Data, AmlCompute, Model, ModelPackage, CodeConfiguration, AzureMLOnlineInferencingServer, ManagedOnlineEndpoint, ManagedOnlineDeployment, Environment
from azure.ai.ml.constants import AssetTypes  
from azure.ai.ml import automl
from azure.identity import DefaultAzureCredential  
import mltable  
import pandas as pd    
import numpy as np    
import random

In [ ]:
# Create MLClient object from configuration file and DefaultAzureCredential  
ml_client = MLClient.from_config(credential=DefaultAzureCredential())  

# Print the MLClient object  
print(ml_client)  

In [ ]:
# Define the name of the CPU compute target to be used or created  
cpu_compute_target = "cpu-cluster"  
  
try:  
    # Try to get the details of the specified compute target if it already exists  
    ml_client.compute.get(cpu_compute_target)  
except Exception:  
    # If the compute target does not exist, catch the exception and create a new one  
    print("Creating a new cpu compute target...")  
    # Define the compute target specifications such as name, VM size, and instance limits  
    compute = AmlCompute(  
        name=cpu_compute_target, size="STANDARD_D2_V2", min_instances=0, max_instances=4  
    )  
    # Initiate the creation or update of the compute target and wait for the operation to complete  
    ml_client.compute.begin_create_or_update(compute).result()  


In [ ]:
from scripts.components import processing_routine

# Pipeline definition - contains a single step using the `processing_routine` component
@pipeline(default_compute=cpu_compute_target)
def pipeline_with_custom_processing(result_count):
    """Sample pipeline with components defined via python function components"""

    # Call component obj as function: apply given inputs & parameters to create a node in pipeline
    run_processing_routine = processing_routine(result_count=result_count)
    
    
# Create an instance of the pipeline and submit a run
pipeline_job = pipeline_with_custom_processing(
    result_count=40
)
    
returned_job = ml_client.jobs.create_or_update(
    pipeline_job, experiment_name="textron_pipeline_sample"
)
returned_job

In [ ]:
# wait until the job completes
ml_client.jobs.stream(returned_job.name)

In [ ]:
from datetime import datetime
from azure.ai.ml.entities import RecurrenceTrigger, RecurrencePattern, JobSchedule
from azure.ai.ml.constants import TimeZone

schedule_name = "Pipeline_Schedule"

schedule_start_time = datetime.utcnow()
recurrence_trigger = RecurrenceTrigger(
    frequency="week",
    interval=1,
    schedule=RecurrencePattern(hours=0, week_days=['sunday'], minutes=[0]),
    start_time=schedule_start_time,
    time_zone=TimeZone.UTC,
)

job_schedule = JobSchedule(
    name=schedule_name, trigger=recurrence_trigger, create_job=pipeline_with_custom_processing(
        result_count=40
    )
)

job_schedule = ml_client.schedules.begin_create_or_update(
    schedule=job_schedule
).result()
print(job_schedule)

In [ ]:
schedules = ml_client.schedules.list()
[s.name for s in schedules]